Отели играют решающую роль в путешествиях, и с расширением доступа к информации появились новые способы выбора лучших.
С помощью этого набора данных, состоящего из 20 тысяч отзывов, полученных с сайта Tripadvisor, вы можете изучить, что делает отель отличным, и, возможно, даже использовать эту модель в своих путешествиях!

[Датасет](https://www.kaggle.com/andrewmvd/trip-advisor-hotel-reviews)  
&copy; Алам, М.Х., Рю, В.-Дж., Ли, С., 2016. Совместное многоплановое отношение к теме: моделирование семантических аспектов для онлайн-обзоров. Информационные науки 339, 206–223.

In [ ]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.metrics import f1_score
#working with text
from sklearn.feature_extraction.text import TfidfVectorizer
#normalizing data
from sklearn.preprocessing import StandardScaler
#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score
#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

In [ ]:
df=pd.read_csv('tripadvisor_hotel_reviews.csv')

In [ ]:
df

Целью модели будет предсказание эмоциональной окраски отзыва:  
  0 - отрицательный(рейтинг 1-3),  
  1  - положительный(рейтинг 4-5)

In [ ]:
def get_sentiment(x):
    if x<=3:
        return 0
    return 1

df['Sentiment']=df['Rating'].apply(get_sentiment)

In [ ]:
df['Sentiment'].value_counts()

Поле Rating можем удалить

In [ ]:
df=df.drop('Rating', axis=1)

Разделим датасет на тренировочную, валидационную и тестовую выборки. И сохраним тестовую выборку на диск

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(df.iloc[:,:-1], df.iloc[:,-1], test_size= 0.2, random_state=24, stratify=df['Sentiment'])
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)

In [ ]:
X_train, X_val, y_train, y_val=train_test_split(X_train, y_train, test_size= 0.25, random_state=24, stratify=y_train)

In [ ]:
import regex
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize

In [ ]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

class FeatImp(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        
    def fit(self, X, y=None):
        self.new_key='lenght'
        return self
    def transform(self, X):
        X[self.new_key]=X[self.key].apply(lambda x: len(x.strip().split()))
        return X
    
class TextLemmatizer(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        
    def fit(self, X, y=None):
        self.stop_words=stopwords.words('english')
        self.lemmatizer=WordNetLemmatizer()
        return self
    
    def transform(self, X):
        X[self.key] = X[self.key].str.replace(r"http\S+", "")
        X[self.key] = X[self.key].str.replace(r"http","")
        X[self.key] = X[self.key].str.replace(r"@/S+","")
        X[self.key] = X[self.key].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
        X[self.key] = X[self.key].str.replace(r"@"," at ")
        X[self.key] = X[self.key].str.lower()
        X[self.key] = X[self.key].apply(self._lemm)
        return X
    
    def _lemm(self, seq):
        # review = regex.sub('[^a-zA-Z]', ' ', seq)    
        review = review.split()    
        review = [word for word in review if not word in set(self.stop_words)]    
        review = [self.lemmatizer.lemmatize(word) for word in review]    
        review = ' '.join(review)
        return review
        

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
text_feat=Pipeline([
    ('lemm', TextLemmatizer(key='Review')),
    ('selector', ColumnSelector(key='Review')),
    ('tfidf', TfidfVectorizer(ngram_range=(1, 3),  max_features=10000, tokenizer = word_tokenize))
    
])
next_feat=Pipeline([    
    ('selector', NumberSelector(key='lenght'))
])

In [ ]:
features=FeatureUnion([
    ('rewiew', text_feat),
    ('lenght', next_feat)
])

In [ ]:
%%time

model = Pipeline([('imp', FeatImp(key='Review')),
    ('feat', features),
    ('classifier', LogisticRegression()),
])

model.fit(X_train, y_train)

In [ ]:
predictions=model.predict_proba(X_val)

In [ ]:
print(classification_report(y_val,np.argmax(predictions,axis=1)))
print(confusion_matrix(y_val, np.argmax(predictions,axis=1)))

In [ ]:
roc_auc_score(y_score=predictions[:, 1][:], y_true=y_val.iloc[:])

Результаты вполне приемлимы, сохраним модель

In [ ]:
with open("model.dill", "wb") as f:
    dill.dump(model, f)